#This files's purpose is to evaluate the performance of the models on different test sets that are trained on 9, 6, and 3 sensors. For the Research question discovering the effect of misalignemnt on decreased number of sensors

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn


drive.mount('/content/drive')

Mounted at /content/drive


# Upload required test set and relevant trained model

In [ ]:
from scipy.io import loadmat

# Upload test set from mat file

dataset = 5  # defines the test set chosen

def define_paths(dataset):
  matfile_path = ""
  model_path = ""
  if dataset == 1:
    matfile_path = "/content/drive/My Drive/PROJECT/REALDISP/REALDISP_SET_1.mat"
    model_path = "/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_1/DCL-3L_Framewise_54_96.pth"
  elif dataset == 2:
    matfile_path = "/content/drive/My Drive/PROJECT/REALDISP/REALDISP_SET_2.mat"
    model_path = "/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_2/DCL-3L_Framewise_54_97.pth"
  elif dataset == 3:
    matfile_path = "/content/drive/My Drive/PROJECT/REALDISP/REALDISP_SET_3.mat"
    model_path = "/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_3/DCL-3L_Framewise_27_97.pth"
  elif dataset == 4:
    matfile_path = "/content/drive/My Drive/PROJECT/REALDISP/REALDISP_SET_4.mat"
    model_path = "/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_4/DCL-3L_Framewise_27_94.pth"
  elif dataset == 5:
    model_path = "/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_FULL/DCL-3L_Framewise_81_99.pth"
    matfile_path = "/content/drive/My Drive/PROJECT/REALDISP/REALDISP_SET_FULL.mat"
  return matfile_path, model_path

matfile_path, model_path = define_paths(dataset)

# Load the .mat file
data = loadmat(matfile_path)

# Extract testData and testLabels
X_test = data['testData']
y_test = data['testLabels'].reshape(-1)

# Extract trainData
X_train = data['trainData']

# Check their shapes
print("testData shape:", X_test.shape)
print("testLabels shape:", y_test.shape)

dict_keys(['__header__', '__version__', '__globals__', 'trainData', 'valData', 'testData', 'trainLabels', 'valLabels', 'testLabels'])
testData shape: (123136, 81)
testLabels shape: (123136,)


# Introducing and setting up the trained model for classification

In [ ]:
class ConvLSTM(nn.Module):
    """Model for human-activity-recognition."""

    def __init__(self, input_channel, num_classes, cnn_channel):
        super().__init__()
        self.n_layers = 2
        self.num_classes = num_classes
        self.n_hidden = 128

        kernal = (5, 1)

        self.features = nn.Sequential(
            nn.Conv2d(input_channel, cnn_channel, kernel_size=kernal),
            nn.GroupNorm(4, cnn_channel),
            nn.MaxPool2d((2, 1)),
            nn.ReLU(),
            nn.Conv2d(cnn_channel, cnn_channel, kernel_size=kernal),
            nn.GroupNorm(4, cnn_channel),
            nn.MaxPool2d((2, 1)),
            nn.ReLU(),
            nn.Conv2d(cnn_channel, cnn_channel, kernel_size=kernal),
            nn.GroupNorm(4, cnn_channel),
            nn.ReLU(),
            # nn.AdaptiveMaxPool2d((4, input_channel))
        )

        self.lstm1 = nn.LSTM(cnn_channel, hidden_size=self.n_hidden, num_layers=self.n_layers)
        self.fc = nn.Linear(self.n_hidden, self.num_classes)
        self.dropout = nn.Dropout()

    def forward(self, x):
        x = x.unsqueeze(1)
        x = x.permute(0, 3, 2, 1)
        x = self.features(x)

        x = x.permute(2, 0, 3, 1)
        x = x.reshape(x.shape[0], x.shape[1], -1)

        # x, _ = self.lstm1(x)
        # x = self.dropout(x)
        # out = self.fc(x[:, -1])

        x = self.dropout(x)
        x, _ = self.lstm1(x)
        x = x[-1, :, :]
        # x = x.view(x.shape[0], -1, 128)
        out = self.fc(x)

        return out


# Create and upload Model

print(f"This is model path:{model_path}")

model = ConvLSTM(X_test.shape[1], num_classes=33, cnn_channel=256)
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

optimizer = Adam(model.parameters(), lr=0.001)

model.load_state_dict(checkpoint['model_state_dict'])

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

This is model path:/content/drive/My Drive/PROJECT/Models/CNNLSTM_DATASET_FULL/DCL-3L_Framewise_81_99.pth


<ipython-input-95-e0c36ca5e664>:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=torch.device('cpu'))


In [ ]:
def drop_rows(sensor_set, sensor_indices, labels, misaligning_row):
    """
    Purpose:
        Simulates sensor misalignment by selectively dropping rows from specific sensors
        in a multi-sensor dataset. This method is used to mimic the effect of slower
        sampling rates or dropped frames in certain sensors.

    Args:
        sensor_set (list of np.ndarray): A list where each element is a sensor's time-series data
                                         (e.g., shape [n_samples, n_features]).
        sensor_indices (list of int): Indices (1-based) of the sensors to misalign by dropping rows.
        labels (list of np.ndarray): A list of label arrays corresponding to each sensor.
        misaligning_row (int): Drop every Nth row to simulate misalignment.

    Returns:
        updated_sensors (list of np.ndarray): Sensors with dropped rows applied to selected indices.
        updated_labels (list of np.ndarray): Corresponding labels with rows dropped to match the data.
    """

    updated_sensors = []
    updated_labels = []

    for i, sensor in enumerate(sensor_set):
        if i + 1 in sensor_indices:  # Check if this sensor should be misaligned (1-based indexing)
            # Create a boolean mask: True for rows to keep, False for every Nth row
            mask = np.ones(sensor.shape[0], dtype=bool)
            mask[::misaligning_row] = False  # Drop every `misaligning_row`th row
            updated_sensors.append(sensor[mask])  # Apply mask to sensor data
            updated_labels.append(labels[i][mask])  # Apply same mask to corresponding labels
        else:
            # Leave sensors not specified in sensor_indices unchanged
            updated_sensors.append(sensor)
            updated_labels.append(labels[i])

    return updated_sensors, updated_labels

def interpolate(sensor_set, sensor_indices, labels, misaligning_row):
    """
    Interpolates missing rows for selected sensors and adds corresponding labels.

    Args:
        sensor_set (list of np.ndarray): List of sensor data arrays, each of shape (frames, axes).
        sensor_indices (list): List of sensor indices that need interpolation.
        labels (np.ndarray): 2D array of shape (frames, num_sensors), containing labels for each frame.
        misaligning_row (int): The interval at which rows should be interpolated.

    Returns:
        tuple: (Updated sensor data list, Updated labels array)
    """

    updated_sensors = []
    updated_labels = []

    for inx, sensor in enumerate(sensor_set):
        sensor_labels = labels[inx]  # Get labels for this sensor
        if inx + 1 in sensor_indices:
            rows, cols = sensor.shape
            interpolated_data = []
            interpolated_labels = []

            for i in range(rows):
                interpolated_data.append(sensor[i])
                interpolated_labels.append(sensor_labels[i])

                # Interpolate every misaligning_row-th frame
                if i % misaligning_row == 0 and i + 1 < rows:
                    interpolated_row = (sensor[i] + sensor[i + 1]) / 2  # Compute interpolated values
                    interpolated_data.append(interpolated_row)
                    interpolated_labels.append(sensor_labels[i])  # Copy previous label

            updated_sensors.append(np.array(interpolated_data))  # Convert to NumPy array
            updated_labels.append(np.array(interpolated_labels))  # Convert labels to NumPy array
        else:
            updated_sensors.append(sensor)  # Keep unchanged sensors
            updated_labels.append(sensor_labels)  # Keep unchanged labels

    # Convert updated_labels list into a stacked NumPy array
    #updated_labels = np.stack(updated_labels, axis=1)  # Convert back to (frames, num_sensors)

    return updated_sensors, updated_labels


def misalign_test_set(sensor_set, labels, num_sensors, misaligning_row=1000):
    """
    Applies misalignment operations on a sensor dataset:
    - Drops rows from 1/3 of the sensors
    - Interpolates missing rows for 1/3 of the sensors
    - Keeps 1/3 of the sensors unchanged

    Args:
        sensor_set (list of np.ndarray): List of sensor arrays (each of shape (frames, axes)).
        labels (np.ndarray): Label array (frames, num_sensors), containing labels per sensor.
        num_sensors (int): The number of sensors in the test set (must be 9, 6, or 3).
        misaligning_row (int): Row misalignment frequency.

    Returns:
        tuple: (Updated X_test, Updated y_test)
    """
    assert num_sensors in [9, 6, 3], "Number of sensors must be 9, 6, or 3"

    # Select indices for each operation
    all_sensors = list(range(num_sensors))
    np.random.shuffle(all_sensors)

    num_to_drop = num_sensors // 3
    num_to_interpolate = num_sensors // 3
    num_to_keep = num_sensors - num_to_drop - num_to_interpolate

    sensor_index_for_drop = all_sensors[:num_to_drop]
    sensor_index_for_inter = all_sensors[num_to_drop:num_to_drop + num_to_interpolate]
    sensor_index_for_keep = all_sensors[num_to_drop + num_to_interpolate:]

    print(f"Drop Sensors idx: {sensor_index_for_drop}")
    print(f"Interpolate Sensors idx: {sensor_index_for_inter}")
    print(f"Keep Sensors idx: {sensor_index_for_keep}")


    # Apply row dropping
    sensor_set, label_set = drop_rows(sensor_set, sensor_index_for_drop, labels, misaligning_row)

    # Apply interpolation
    sensor_set, label_set = interpolate(sensor_set, sensor_index_for_inter, label_set, misaligning_row)

    # Trim all sensors to the same length
    min_row = min(sensor.shape[0] for sensor in sensor_set)
    for i in range(len(sensor_set)):
        sensor_set[i] = sensor_set[i][:min_row]
        label_set[i] = label_set[i][:min_row]

    # Majority voting for final labels
    stacked_labels = np.stack(label_set, axis=0)
    y_test = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=stacked_labels.astype(int))

    # Concatenate sensor data into final feature array
    X_test = np.concatenate(sensor_set, axis=1)

    return X_test, y_test

Chain test data 9 times to make lenght of 6 hours

In [ ]:
X_test = np.tile(X_test, (9, 1))
y_test = np.tile(y_test, 9)

print(X_test.shape)
print(y_test.shape)

(1108224, 81)
(1108224,)


In [ ]:
# Define the number of features per sensor (e.g., 3-axis accelerometer + 3-axis gyro + 3-axis magnetometer)
sensor_length = 9

# Determine the total number of sensors based on the shape of X_test
num_sensors = X_test.shape[1] // sensor_length
print("Number of Sensors is: ",num_sensors)

# Purpose:
# Splits the full test dataset (X_test) into a list of individual sensor data arrays.
# Each array corresponds to a sensor and contains only its 9-axis data.
sensor_set = [X_test[:, i * sensor_length:(i + 1) * sensor_length] for i in range(num_sensors)]

# Creates a corresponding label array for each sensor
# Each sensor receives a copy of the original labels (y_test)
label_set = np.tile(y_test[:, np.newaxis], num_sensors).T

# Applies misalignment to the test set using a custom function
# This function is expected to return the misaligned X_test and y_test
X_test, y_test = misalign_test_set(sensor_set, label_set, num_sensors)

# Output the shapes of the final datasets for confirmation
print("Final X_test shape:", X_test.shape)
print("Final y_test shape:", y_test.shape)

#Normalising data

In [ ]:
mean_train = np.mean(X_train, axis=0)
std_train = np.std(X_train, axis=0)
X_test = (X_test - mean_train) / std_train

#Segmenting data (Windowing)

In [ ]:
from numpy.lib.stride_tricks import as_strided as ast
from scipy.stats import mode
from collections import Counter

import numpy as np
from numpy.lib.stride_tricks import as_strided as ast
# from scipy.stats import mode  # Uncomment if using mode-based labeling

def norm_shape(shape):
    """
    Normalizes input shape into a tuple.
    If an integer is passed, converts it to a 1-element tuple.
    Raises an error if the input is neither an int nor iterable.
    """
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        pass
    try:
        t = tuple(shape)
        return t
    except TypeError:
        pass
    raise TypeError('shape must be an int, or a tuple of ints')

def sliding_window(a, ws, ss=None, flatten=True):
    """
    Applies a sliding window over a NumPy array.

    Args:
        a (np.ndarray): Input array.
        ws (int or tuple): Window size.
        ss (int or tuple): Step size. If None, defaults to window size (no overlap).
        flatten (bool): If True, output is reshaped to 2D.

    Returns:
        np.ndarray: An array of windowed slices from the input.
    """
    if ss is None:
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)
    shape = np.array(a.shape)

    # Ensure compatible dimensions
    ls = [len(shape), len(ws), len(ss)]
    if len(set(ls)) != 1:
        raise ValueError('a.shape, ws and ss must all have the same length. They were %s' % str(ls))

    if np.any(ws > shape):
        raise ValueError('ws cannot be larger than a in any dimension. a.shape: %s, ws: %s' % (str(a.shape), str(ws)))

    # Compute output shape and strides
    newshape = norm_shape(((shape - ws) // ss) + 1) + norm_shape(ws)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides

    strided = ast(a, shape=newshape, strides=newstrides)

    if not flatten:
        return strided

    # Reshape into flat list of slices
    meat = len(ws) if ws.shape else 0
    firstdim = (np.prod(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + (newshape[-meat:])
    return strided.reshape(dim)

def data_sliding_window(data_x, data_y, ws, ss):
    """
    Applies a sliding window to both input features and labels.

    Args:
        data_x (np.ndarray): Feature data of shape [n_samples, n_features].
        data_y (np.ndarray): Corresponding labels of shape [n_samples].
        ws (int): Window size (number of frames per window).
        ss (int): Step size (window shift amount).

    Returns:
        data_x_windowed (np.ndarray): Windowed feature data of shape [n_windows, ws, n_features].
        data_y_windowed (np.ndarray): One label per window (default: last label in window).
    """
    single_label_windows = []

    # Apply sliding window on features
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))

    # Apply sliding window on labels
    #data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])

    # for mode-based labeling, use the line below instead
    data_y = np.asarray([[mode(i)[0]] for i in sliding_window(data_y, ws, ss)])

    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# -------------------------
# Execution of the windowing on test data

window_size = 128       # Size of each window (frames)
step_size = 64          # Step size (overlap of 50%)

# Apply the sliding window
X_test_windows, y_test_windows = data_sliding_window(X_test, y_test, window_size, step_size)

# Output shapes for confirmation
print("X_test_windows shape:", X_test_windows.shape)  # Expected shape: [n_windows, window_size, n_features]
print("y_test_windows shape:", y_test_windows.shape)  # Expected shape: [n_windows]

<ipython-input-99-7006695be825>:76: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
<ipython-input-99-7006695be825>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  i = int(shape)
<ipython-input-99-7006695be825>:78: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  data_y = np.asarray([[mode(i)[0]] for i in sliding_window(data_y, ws, ss)])  # Choose most frequent label


X_test_windows shape: (34631, 64, 81)
y_test_windows shape: (34631,)


#Converts preprocessed test data (windowed X_test and y_test) into PyTorch tensors, wraps them into a DataLoader, performs batch-wise prediction using a trained model, and collects the predicted and true labels for evaluation.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

X_test_torch = torch.tensor(X_test_windows, dtype=torch.float32)
y_test_torch = torch.tensor(y_test_windows, dtype=torch.long)

# Create a DataLoader
test_dataset = TensorDataset(X_test_torch, y_test_torch)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:  # Retrieve both features and labels
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.numpy())
        true_labels.extend(labels.numpy())

predictions = np.array(predictions) + 1

predictions_half = predictions[int(len(predictions)/2):]
true_labels_half = true_labels[int(len(true_labels)/2):]

predictions_quat = predictions[int(len(predictions)*0.75):]
true_labels_quat = true_labels[int(len(true_labels)*0.75):]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def per_class_accuracy(true_labels, predictions):
    """
    Computes accuracy for each individual class.
    """
    unique_classes = np.unique(true_labels)  # Get all unique class labels
    class_accuracies = {}

    for cls in unique_classes:
        # Filter predictions and true labels for the current class
        mask = true_labels == cls
        class_acc = accuracy_score(true_labels[mask], predictions[mask])
        class_accuracies[cls] = class_acc  # Store per-class accuracy

    return class_accuracies

def evaluate_top_bottom_accuracies(true_labels, predictions, name="Predictions"):
    """
    Evaluates:
    - Accuracy for overall data
    - Accuracy for top 5 classes (most correctly predicted)
    - Accuracy for bottom 5 classes (least correctly predicted)
    - Mean F1-score
    - F1-score Standard Deviation
    """

    print(f"\n=== Evaluating {name} ===")

    # Compute overall accuracy
    overall_accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {overall_accuracy:.4f}")

    # Compute per-class accuracy
    class_accuracies = per_class_accuracy(true_labels, predictions)

    # Sort classes by accuracy
    sorted_classes = sorted(class_accuracies.items(), key=lambda x: x[1], reverse=True)

    # Extract top 5 and bottom 5 classes
    top_5_classes = [cls for cls, _ in sorted_classes[:5]]
    bottom_5_classes = [cls for cls, _ in sorted_classes[-5:]]

    print(f"Top 5 Classes: {top_5_classes}")
    print(f"Bottom 5 Classes: {bottom_5_classes}")

    # Filter predictions for top 5 and bottom 5 classes
    top5_mask = np.isin(true_labels, top_5_classes)
    bottom5_mask = np.isin(true_labels, bottom_5_classes)

    # Compute accuracy for top 5 and bottom 5 activities
    accuracy_top5 = accuracy_score(true_labels[top5_mask], predictions[top5_mask])
    accuracy_bottom5 = accuracy_score(true_labels[bottom5_mask], predictions[bottom5_mask])

    print(f"Top 5 Accuracy: {accuracy_top5:.4f}")
    print(f"Bottom 5 Accuracy: {accuracy_bottom5:.4f}")

    # Compute Mean F1-score and F1-score Standard Deviation
    f1_scores = f1_score(true_labels, predictions, average=None)  # Get F1-score for each class
    mean_f1 = np.mean(f1_scores)  # Compute mean F1-score
    std_f1 = np.std(f1_scores)  # Compute standard deviation of F1-scores

    print(f"Mean F1-score: {mean_f1:.4f}")
    print(f"F1-score Standard Deviation: {std_f1:.4f}")

    return {
        "overall_accuracy": overall_accuracy,
        "top_5_classes": top_5_classes,
        "bottom_5_classes": bottom_5_classes,
        "top_5_accuracy": accuracy_top5,
        "bottom_5_accuracy": accuracy_bottom5,
        "mean_f1": mean_f1,
        "std_f1": std_f1
    }


results_half = evaluate_top_bottom_accuracies(
    np.array(true_labels), np.array(predictions),
    name="Non-Augmented Predictions (Full)"
)

results_half = evaluate_top_bottom_accuracies(
    np.array(true_labels_half), np.array(predictions_half),
    name="Non-Augmented Predictions (Second Half)"
)

results_half = evaluate_top_bottom_accuracies(
    np.array(true_labels_quat), np.array(predictions_quat),
    name="Non-Augmented Predictions (Last Quarter)"
)


=== Evaluating Non-Augmented Predictions (Full) ===
Overall Accuracy: 0.8346
Top 5 Classes: [27, 31, 33, 26, 29]
Bottom 5 Classes: [1, 2, 5, 4, 14]
Top 5 Accuracy: 0.9705
Bottom 5 Accuracy: 0.6804
Mean F1-score: 0.8264
F1-score Standard Deviation: 0.0943

=== Evaluating Non-Augmented Predictions (Second Half) ===
Overall Accuracy: 0.8348
Top 5 Classes: [27, 33, 31, 26, 29]
Bottom 5 Classes: [3, 2, 5, 4, 14]
Top 5 Accuracy: 0.9718
Bottom 5 Accuracy: 0.6756
Mean F1-score: 0.8264
F1-score Standard Deviation: 0.0943

=== Evaluating Non-Augmented Predictions (Last Quarter) ===
Overall Accuracy: 0.8358
Top 5 Classes: [27, 31, 33, 26, 29]
Bottom 5 Classes: [1, 2, 5, 4, 14]
Top 5 Accuracy: 0.9719
Bottom 5 Accuracy: 0.6860
Mean F1-score: 0.8281
F1-score Standard Deviation: 0.0914
